In [1]:
import numpy as np
import cs_vqe as c
import ast
import os

from openfermion import qubit_operator_sparse
import conversion_scripts as conv_scr
import scipy as sp

In [2]:
from openfermion import qubit_operator_sparse
import conversion_scripts as conv_scr
from openfermion.ops import QubitOperator

In [3]:
# with open("hamiltonians.txt", 'r') as input_file:
#     hamiltonians = ast.literal_eval(input_file.read())
    
working_dir = os.getcwd()
data_dir = os.path.join(working_dir, 'data')
data_hamiltonians_file = os.path.join(data_dir, 'hamiltonians.txt')


with open(data_hamiltonians_file, 'r') as input_file:
    hamiltonians = ast.literal_eval(input_file.read())

In [4]:
for key in hamiltonians.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

H2-S1_STO-3G_singlet          n_qubits:  18   
C1-O1_STO-3G_singlet          n_qubits:  16   
H1-Cl1_STO-3G_singlet         n_qubits:  16   
H1-Na1_STO-3G_singlet         n_qubits:  16   
H2-Mg1_STO-3G_singlet         n_qubits:  17   
H1-F1_3-21G_singlet           n_qubits:  18   
H1-Li1_3-21G_singlet          n_qubits:  18   
Be1_STO-3G_singlet            n_qubits:  5    
H1-F1_STO-3G_singlet          n_qubits:  8    
H1-Li1_STO-3G_singlet         n_qubits:  8    
Ar1_STO-3G_singlet            n_qubits:  13   
F2_STO-3G_singlet             n_qubits:  15   
H1-O1_STO-3G_singlet          n_qubits:  8    
H2-Be1_STO-3G_singlet         n_qubits:  9    
H2-O1_STO-3G_singlet          n_qubits:  10   
H2_3-21G_singlet              n_qubits:  5    
H2_6-31G_singlet              n_qubits:  5    
H3-N1_STO-3G_singlet          n_qubits:  13   
H4-C1_STO-3G_singlet          n_qubits:  14   
Mg1_STO-3G_singlet            n_qubits:  13   
N2_STO-3G_singlet             n_qubits:  15   
Ne1_STO-3G_si

In [5]:
# mol_key = 'H2_6-31G_singlet'  
# mol_key ='H2-O1_STO-3G_singlet'
# mol_key = 'H1-He1_3-21G_singlet_1+'
mol_key = 'H3_STO-3G_singlet_1+'

# currently index 2 is contextual part
# ''''''''''''''''3 is NON contextual part

# join together for full Hamiltonian:
ham = hamiltonians[mol_key][2]
ham.update(hamiltonians[mol_key][3]) # full H 
ham

{'III': -1.7512307459285525,
 'IIX': -0.023568152336180023,
 'IIZ': -0.017109477140260287,
 'IXI': 0.02356815233618002,
 'IXZ': 0.02356815233617983,
 'IYY': -0.07195737217001562,
 'IZI': -0.017109477140260287,
 'IZX': -0.023568152336179825,
 'IZZ': 0.31270210682950855,
 'XII': 0.01872992170537467,
 'XIX': 0.023568139980123585,
 'XIZ': 0.01872992170537467,
 'XXI': -0.023568139980123585,
 'XXX': 0.03597868636603963,
 'XXZ': -0.023568139980123585,
 'XYY': -0.03597868636603963,
 'XZI': 0.01872992170537467,
 'XZX': 0.023568139980123585,
 'XZZ': 0.01872992170537467,
 'YIY': 0.023568139980123585,
 'YXY': 0.03597868636603963,
 'YYI': -0.023568139980123585,
 'YYX': 0.03597868636603963,
 'YYZ': -0.023568139980123585,
 'YZY': 0.023568139980123585,
 'ZII': -0.45436486525596403,
 'ZIX': -0.023568152336180023,
 'ZIZ': 0.37110605476609787,
 'ZXI': 0.02356815233618002,
 'ZXZ': 0.02356815233617983,
 'ZYY': -0.07195737217001562,
 'ZZI': 0.37110605476609804,
 'ZZX': -0.023568152336179825,
 'ZZZ': -0.2878

In [6]:
print(f"n_qubits:  {hamiltonians[mol_key][1]}")

n_qubits:  3


# Get non-contextual H

In [7]:
nonH_guesses = c.greedy_dfs(ham, 10, criterion='weight')

nonH = max(nonH_guesses, key=lambda x:len(x)) # largest nonCon part found by dfs alg

Split into:

$$H = H_{c} + H_{nc}$$

In [8]:
nonCon_H = {}
Con_H = {}

for P in ham:
    if P in nonH:
        nonCon_H[P]=ham[P]
    else:
        Con_H[P]=ham[P]
        

## Testing contextuality

In [9]:
print('Is NONcontextual correct:', not c.contextualQ_ham(nonCon_H))
print('Is contextual correct:',c.contextualQ_ham(Con_H))

Is NONcontextual correct: True
Is contextual correct: True


# Classical part of problem!

Take $H_{nc}$ and split into:
- $Z$ = operators that completely comute with all operators in $S$
- $T$ = remaining operators in $S$
    - where $S = Z \cup T$  and $S$ is set of Pauli operators in $H_{nc}$
    
    
- We then split the set $T$ into cliques $C_{1}, C_{2}, ... , C_{|T|}$
    - all ops in a clique commute
    - ops between cliques anti-commute!

In [10]:
bool_flag, Z_list, T_list = c.contextualQ(list(nonCon_H.keys()), verbose=True)

In [11]:
Z_list

['III', 'IZZ', 'ZIZ', 'ZZI']

In [12]:
T_list

['IIZ', 'IZI', 'XXX', 'XYY', 'YXY', 'YYX', 'ZII', 'ZZZ']

## Get quasi model

First we define

- $C_{i1}$ = first Pauli in each $C_{i}$ set
- $A_{ij} = C_{ij}C_{1i}$


- $G^{prime} = \{1 P_{i} \;| \; i=1,2,...,|Z| \}$
    - aka all the completely commuting terms with coefficients set to +1!

- We define G to be an independent set of $G^{prime}$
    - where $G \subseteq G^{prime}$


In [13]:
G_list, Ci1_list, all_mappings = c.quasi_model(nonCon_H)

In [14]:
print('non-independent Z list:', Z_list)
print('G (independent) Z list:', G_list)

non-independent Z list: ['III', 'IZZ', 'ZIZ', 'ZZI']
G (independent) Z list: ['ZIZ', 'IZZ']


In [15]:
print('all Ci1 terms:', Ci1_list)

all Ci1 terms: ['IIZ', 'XXX']


$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

In [16]:
# Assemble all the mappings from terms in the Hamiltonian to their products in R:
all_mappings

{'III': [[], [], 1],
 'IIZ': [[], ['IIZ'], 1],
 'IZI': [['IZZ'], ['IIZ'], 1],
 'IZZ': [['IZZ'], [], 1],
 'XXX': [[], ['XXX'], 1],
 'XYY': [['IZZ'], ['XXX'], (-1+0j)],
 'YXY': [['ZIZ'], ['XXX'], (-1+0j)],
 'YYX': [['ZIZ', 'IZZ'], ['XXX'], (-1+0j)],
 'ZII': [['ZIZ'], ['IIZ'], 1],
 'ZIZ': [['ZIZ'], [], 1],
 'ZZI': [['ZIZ', 'IZZ'], [], 1],
 'ZZZ': [['ZIZ', 'IZZ'], ['IIZ'], 1]}

Overall $R$ is basically reduced non-contextual set
- where everything in original non-contextual set can be found by **inference!**

# Function form

$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

- note q to do with $G$
- note r to do with $C_{i1}$

In [17]:
model = [G_list, Ci1_list, all_mappings]

fn_form = c.energy_function_form(nonCon_H, model)

# returns [
#            denstion of q,
#            dimension of r,
#            [coeff, indices of q's, indices of r's, term in Hamiltonian]
#         ]


In [18]:
fn_form

[2,
 2,
 [[-1.7512307459285525, [], [], 'III'],
  [-0.017109477140260287, [], [0], 'IIZ'],
  [-0.017109477140260287, [1], [0], 'IZI'],
  [0.31270210682950855, [1], [], 'IZZ'],
  [0.03597868636603963, [], [1], 'XXX'],
  [(0.03597868636603963-0j), [1], [1], 'XYY'],
  [(-0.03597868636603963+0j), [0], [1], 'YXY'],
  [(-0.03597868636603963+0j), [0, 1], [1], 'YYX'],
  [-0.45436486525596403, [0], [0], 'ZII'],
  [0.37110605476609787, [0], [], 'ZIZ'],
  [0.37110605476609804, [0, 1], [], 'ZZI'],
  [-0.2878474382772282, [0, 1], [0], 'ZZZ']]]

In [19]:
Energy_function = c.energy_function(fn_form)

In [20]:
import random

### now for the q terms we only have +1 or -1 assignment!
q_variables = [random.choice([1,-1]) for _ in range(fn_form[0])]


### r variables is anything that makes up unit vector!
r_variables = c.angular(np.arange(0,2*np.pi, fn_form[1]))
r_variables

(1.0, -0.0, -0.0, -0.0, 0.0)

In [21]:
 Energy_function(*q_variables,*r_variables)

-2.230450279736797

find_gs_nonconfunction optimizes above steps by:
1. brute forcing all choices of ```q_variables```
    - ```itertools.product([1,-1],repeat=fn_form[0])```
2. optimizing over ```r_variables``` (in code ```x```)
    - using SciPy optimizer!

In [22]:
model = [G_list, Ci1_list, all_mappings]

lowest_eigenvalue, ground_state_params, model_copy, fn_form_copy,  = c.find_gs_noncon(nonCon_H,
               method = 'differential_evolution',
               model=model,
               fn_form=fn_form) # returns:  best + [model, fn_form]

print(lowest_eigenvalue)
print(ground_state_params)

-2.903212918716725
[[-1, 1], [-0.9799593376325533, -0.19919763198082302]]


In [23]:
## check
Energy_function(*ground_state_params[0],*ground_state_params[1]) == lowest_eigenvalue

True

# Now need to rotate Hamiltonian!

We now have non contextual ground state: $(\vec{q}, \vec{r})$

In [24]:
ground_state_params

[[-1, 1], [-0.9799593376325533, -0.19919763198082302]]

We can use this result - ground state of $H_{nc}$ -  as a classical estiamte of our ground state of the full Hamiltonian ($H = H_{c} + H_{nc}$)

However we can also obtain a quantum correction using $H_{c}$

By minimizing theenergy of the remaining terms in the Hamiltonian over the quantum states that are **consistent with the noncon-textual ground state**.

To do this we first rotate each $G_{j}$ and $\mathcal{A} = \sum_{i=1}^{N} r_{i}A_{i}$:

In [25]:
model = [G_list, Ci1_list, all_mappings]

print(G_list) # G_j terms!
print(Ci1_list) # mathcal(A)

['ZIZ', 'IZZ']
['IIZ', 'XXX']


to SINGLE QUBIT pauli Z operators!

- to map the operators in $G$ to single qubit Pauli operators, we use $\frac{\pi}{2}$ rotations!

- note $\mathcal{A}$ is an anti-commuting set... therefore we can use $N-1$ rotations as in unitary partitioning's sequence of rotations to do this!
    - $R^{\dagger}\mathcal{A} R = \text{single Pauli op}$

# Rotate full Hamiltonian to basis with diagonal noncontextual generators!

function ```diagonalize_epistemic```:
1. first if else statement:
    - if cliques present:
        - first maps A to single Pauli operator (if cliques present)
        - then rotates to diagonlize G union with single Pauli opator of A (hence GuA name!)
    - else if NO cliques present:
        - gets rotations to diagonlize G
        
     - these rotations make up GuA term in code!
2. NEXT code loops over terms in GuA (denoted as g in code)
    - if g is not a single qubit $Z$:
        - code generates code to rotate operator to make g diagonal (rotations)
        - then constructs map of g to single Z (J rotation)
    - Note R is applied to GuA
    
    
#########
- Note rotations are given in Appendix A of https://arxiv.org/pdf/2011.10027.pdf
    - First code checks if g op in GuA is diagonal
        - if so then needs to apply "K" rotation (involving $Y$ and $I$ operators (see pg 11 top) to make it NOT diagononal
    - now operator will be diagnoal!
    - next generate "J" rotation
        - turns non-diagonal operator into a single qubit $Z$ operator!

In [26]:
# Get sequence of rotations requried to diagonalize the generators for the noncontextual ground state!

Rotations_list, diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops = c.diagonalize_epistemic(model,
                                                                                                            fn_form,
                                                                                                            ground_state_params)

In [27]:
# rotations to map A to single Pauli operator!
Rotations_list

[[3.342131729291596, 'XXY'],
 ['pi/2', 'YII'],
 ['pi/2', 'YIZ'],
 ['pi/2', 'IYI'],
 ['pi/2', 'IYZ']]

In [28]:
# rotations to diagonlize G
diagonalized_generators_GuA

['ZII', 'IZI', 'IIZ']

In [29]:
eigen_vals_nonC_ground_state_GuA_ops

array([ 1., -1.,  1.])

In [30]:
# for rotation in Rotations_list:
#     for p in ham:
#         if not c.commute(rotation[1],p):
#             print(rotation)
#     print()

In [31]:
model = [G_list, Ci1_list, all_mappings]
ham_noncon = hamiltonians[mol_key][3]

lowest_eigenvalue, ground_state_params, model_copy, fn_form_copy,  = c.find_gs_noncon(
               ham_noncon,#nonCon_H,
               method = 'differential_evolution',
               model=model,
               fn_form=fn_form) # returns:  best + [model, fn_form]

# Will's method

In [32]:
ham_noncon = hamiltonians[mol_key][3]
n_qubits = hamiltonians[mol_key][1]
true_gs= hamiltonians[mol_key][4]

true_gs, approxs_out, errors_out, order_out = c.csvqe_approximations_heuristic(ham,
                                                                                   ham_noncon,
                                                                                   n_qubits, 
                                                                                   true_gs)
order_out

[0, 2, 1]

In [33]:
errors_out

[0.012805571551730122,
 0.0003080609044734395,
 -1.3322676295501878e-15,
 8.881784197001252e-16]

In [34]:
### old way
# order = list(range(hamiltonians[mol_key][1]))
order = order_out
red_H = c.get_reduced_hamiltonians(ham,
                           model,
                           fn_form,
                           ground_state_params,
                           order)
len(red_H)

4

In [35]:
list(range(hamiltonians[mol_key][1]))

[0, 1, 2]

In [36]:
red_H[2]

{'II': -1.438528639099044,
 'IZ': 0.01919944551942593,
 'ZX': 0.07733163398184689,
 'YY': -0.06305338666771393,
 'XI': 0.051042861476446974,
 'XZ': -0.03745984341074934,
 'IX': -0.07733163397053659,
 'ZZ': -0.7416716156049102,
 'ZI': -0.7422121095321959,
 'XX': 0.07195737217001562}

In [37]:
approxs_out

[-2.903212918716724,
 -2.9157104293639806,
 -2.9160184902684554,
 -2.916018490268453]

# MY way  - SeqRot

In [38]:
import cs_vqe_with_SeqRot as c_SeqRot

In [39]:
ham_noncon = hamiltonians[mol_key][3]
n_qubits = hamiltonians[mol_key][1]
true_gs= hamiltonians[mol_key][4]

true_gs, approxs_out, errors_out, order_out = c_SeqRot.csvqe_approximations_heuristic_SeqRot(ham,
                                                                                   ham_noncon,
                                                                                   n_qubits, 
                                                                                   true_gs)

from copy import deepcopy 
updated_order = deepcopy(order_out)
updated_order

[0, 2, 1]

In [40]:
errors_out

[0.012805571551729678, 0.0003080609044729954, 0.0, 2.6645352591003757e-15]

In [41]:

red_H_SeqRot = c_SeqRot.get_reduced_hamiltonians_SeqRot(ham,
                           model,
                           fn_form,
                           ground_state_params,
                           updated_order,
                          check_reduction=True)#order)
len(red_H_SeqRot)

4

In [42]:
red_H_SeqRot[2]

{'II': -1.4385286390990442,
 'IZ': 0.019199445519425935,
 'ZX': 0.0773316339818469,
 'YY': -0.06305338666771394,
 'XI': 0.05104286147644698,
 'XZ': -0.037459843410749344,
 'IX': -0.07733163397053659,
 'ZZ': -0.7416716156049104,
 'ZI': -0.7422121095321959,
 'XX': 0.07195737217001562}

In [43]:
red_H[2]

{'II': -1.438528639099044,
 'IZ': 0.01919944551942593,
 'ZX': 0.07733163398184689,
 'YY': -0.06305338666771393,
 'XI': 0.051042861476446974,
 'XZ': -0.03745984341074934,
 'IX': -0.07733163397053659,
 'ZZ': -0.7416716156049102,
 'ZI': -0.7422121095321959,
 'XX': 0.07195737217001562}

## Check energies

In [44]:
hamilt_ind = 0
# red_H[hamilt_ind]

In [45]:
import conversion_scripts as conv_scr

H = conv_scr.Get_Openfermion_Hamiltonian(red_H[hamilt_ind])
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
# sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
min(np.linalg.eigvalsh(sparseH.todense()))

-2.903212918716725

In [46]:
H = conv_scr.Get_Openfermion_Hamiltonian(red_H_SeqRot[hamilt_ind])
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
# sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
min(np.linalg.eigvalsh(sparseH.todense()))

-2.903212918716725

In [47]:
H = conv_scr.Get_Openfermion_Hamiltonian(ham)
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
# sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
min(np.linalg.eigvalsh(sparseH.todense()))

-2.916018490268456

# MY way  - LCU

In [48]:
import cs_vqe_with_LCU as c_LCU

In [49]:
ham_noncon = hamiltonians[mol_key][3]
n_qubits = hamiltonians[mol_key][1]
true_gs= hamiltonians[mol_key][4]
N_index=0
gs_true, approxs_out, errors_out, order_out = c_LCU.csvqe_approximations_heuristic_LCU(ham, 
                                                                                       ham_noncon, 
                                                                                       n_qubits,
                                                                                       true_gs,
                                                                                       check_reduction=True)

from copy import deepcopy 
updated_order = deepcopy(order_out)
updated_order

[0, 2, 1]

In [50]:
approxs_out

[-2.9032129187167244,
 -2.9157104293639806,
 -2.916018490268454,
 -2.916018490268453]

In [51]:
nonCon_H == hamiltonians[mol_key][3] # something odd here!

True

In [52]:
model = [G_list, Ci1_list, all_mappings]

lowest_eigenvalue, ground_state_params, model_copy, fn_form_copy,  = c.find_gs_noncon(nonCon_H,
               method = 'differential_evolution',
               model=model,
               fn_form=fn_form) # returns:  best + [model, fn_form]

print(lowest_eigenvalue)
print(ground_state_params)

-2.903212918716725
[[-1, 1], [-0.9799593376325533, -0.19919763198082302]]


In [53]:

red_H_LCU = c_LCU.get_reduced_hamiltonians_LCU(ham,
                           model,
                           fn_form,
                           ground_state_params,
                           updated_order,
                          check_reduction=True)#order)
len(red_H_LCU)

4

In [54]:
H = conv_scr.Get_Openfermion_Hamiltonian(red_H_LCU[1])
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
# sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
min(np.linalg.eigvalsh(sparseH.todense()))

-2.903275084812363

# check H of different methods

In [56]:
red_H_LCU[-2]

{'II': -1.438528639099044,
 'IZ': 0.019199445519425987,
 'ZX': 0.07733163398184688,
 'YY': -0.06305338666771397,
 'XI': 0.05104286147644694,
 'XZ': -0.037459843410749344,
 'IX': -0.07733163397053612,
 'ZZ': -0.7416716156049104,
 'ZI': -0.7422121095321959,
 'XX': 0.07195737217001562}

In [57]:
red_H_SeqRot[-2]

{'II': -1.4385286390990442,
 'IZ': 0.019199445519425935,
 'ZX': 0.0773316339818469,
 'YY': -0.06305338666771394,
 'XI': 0.05104286147644698,
 'XZ': -0.037459843410749344,
 'IX': -0.07733163397053659,
 'ZZ': -0.7416716156049104,
 'ZI': -0.7422121095321959,
 'XX': 0.07195737217001562}

In [58]:
red_H[-2] # Will's way

{'II': -1.438528639099044,
 'IZ': 0.01919944551942593,
 'ZX': 0.07733163398184689,
 'YY': -0.06305338666771393,
 'XI': 0.051042861476446974,
 'XZ': -0.03745984341074934,
 'IX': -0.07733163397053659,
 'ZZ': -0.7416716156049102,
 'ZI': -0.7422121095321959,
 'XX': 0.07195737217001562}